## Imports

In [3]:
import re
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

## Utils

In [12]:
def parse_performance_data(text):
    pattern = re.compile(
        r"Algorithm:\s*(?P<Algorithm>\S+)\s*"
        r"Instance:\s*(?P<Instance>\S+)\s*"
        r"Initial Score:\s*(?P<Initial_Score>\d+)\s*"
        r"Score:\s*(?P<Score>\d+)\s*"
        r"Number\s*of\s*Evaluations:\s*(?P<Number_of_Evaluations>\d+)\s*"
        r"Number\s*of\s*Performed\s*Moves:\s*(?P<Number_of_Performed_Moves>\d+)\s*"
        r"Number\s*of\s*Best\s*Solution\s*Updates:\s*(?P<Number_of_Best_Solution_Updates>\d+)\s*"
        r"Solution:\s*(?P<Solution>[\d\s]+)\s*"
        r"Optimal\s*Score:\s*(?P<Optimal_Score>\d+)\s*"
        r"Optimal\s*Solution:\s*(?P<Optimal_Solution>[\d\s]+)"
    )
    
    rows = []
    for match in pattern.finditer(text):
        row = match.groupdict()
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    numerical_cols = ['Initial_Score', 'Score', 'Number_of_Evaluations', 'Number_of_Performed_Moves', 'Number_of_Best_Solution_Updates', 'Optimal_Score']
    df[numerical_cols] = df[numerical_cols].apply(pd.to_numeric)
    
    return df

def parse_runtime_data(text):
    pattern = re.compile(
        r"Algorithm:\s*(?P<Algorithm>\S+)\s*"
        r"Instance:\s*(?P<Instance>\S+)\s*"
        r"Runtime:\s*(?P<Runtime>\S+)\s*"
    )
    
    rows = []
    for match in pattern.finditer(text):
        row = match.groupdict()
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    numerical_cols = ['Runtime']
    df[numerical_cols] = df[numerical_cols].apply(pd.to_numeric)
    
    return df

## Read Data

In [6]:
with open('performance_results.txt', 'r') as file:
    data = file.read()

df = parse_performance_data(data)

In [11]:
df.tail()

,Algorithm,Instance,Initial_Score,Score,Number_of_Evaluations,Number_of_Performed_Moves,Number_of_Best_Solution_Updates,Solution,Optimal_Score,Optimal_Solution
10,steepestLS,bur26a,5841774,5458470,7800,23,23,8 14 10 5 7 22 3 2 11 6 20 0 18 13 1 12 19 4 1...,5426670,11 15 26 7 4 13 12 2 6 18 9 5 1 21 8 14 3 20 1...
11,steepestLS,bur26a,5990530,5450785,5850,17,17,12 14 2 22 25 10 6 3 1 11 17 4 0 13 5 20 8 18 ...,5426670,11 15 26 7 4 13 12 2 6 18 9 5 1 21 8 14 3 20 1...
12,steepestLS,bur26a,5852743,5439529,7475,22,22,3 10 0 14 7 22 12 5 15 1 18 8 4 20 6 13 17 11 ...,5426670,11 15 26 7 4 13 12 2 6 18 9 5 1 21 8 14 3 20 1...
13,rsteepestLS,bur26a,5990530,5450785,5850,17,17,12 14 2 22 25 10 6 3 1 11 17 4 0 13 5 20 8 18 ...,5426670,11 15 26 7 4 13 12 2 6 18 9 5 1 21 8 14 3 20 1...
14,rsteepestLS,bur26a,5953351,5440232,6500,19,19,6 10 14 3 7 22 2 12 11 20 0 8 4 18 5 1 19 13 1...,5426670,11 15 26 7 4 13 12 2 6 18 9 5 1 21 8 14 3 20 1...


In [15]:
with open('runtime_results.txt', 'r') as file:
    data = file.read()

df = parse_runtime_data(data)

In [16]:
df.head()

,Algorithm,Instance,Runtime
0,heuristic,bur26a,7117.0
1,antiheuristic,bur26a,7455.0
2,random,bur26a,1032.0
3,greedyLS,bur26a,4403190.0
4,steepestLS,bur26a,2726280.0


## Analysis

In [18]:
df[(df.Algorithm == "greedyLS")]["Runtime"].mean()

3308546.6666666665